In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from category_encoders import TargetEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.neural_network import MLPRegressor

## function

In [2]:
def nanfill(data,row, column):
    tag = data.iloc[row,1]
    # print(data[data.iloc[:,1] == tag].iloc[:,column],'\n')
    temp = data[data.iloc[:,1] == tag].iloc[:,column]
    temp = np.average(temp[temp.notna()])
    if (pd.isna(temp)):
        print(row,column)
    data.iloc[row,column] = temp

# def normalization(data):
#     for i in range(data.shape[1]):
#         m1 = min(data.iloc[:,i])
#         m2 = max(data.iloc[:,i])
#         if m1 != m2:
#             data.iloc[:,i] = (data.iloc[:,i] - m1) / (m2 - m1)
#         else:
#             return -1

def normalization(data):
    for i in range(data.shape[1]):
        avg = data.iloc[:,i].mean()
        std = data.iloc[:,i].std(ddof = 1)
        data.iloc[:,i] = (data.iloc[:,i] - avg) / std
        
# 计算交叉验证结果的均值和标准差， 默认为五折交叉验证，结果为相关系数R2      
def cross_validate(models, cv = 5, scoring = 'r2'):
    means = []
    stds = []
    for model in models:
        score = cross_val_score(model, X_train, y_train, cv = cv, scoring = scoring)
        means.append(score.mean())
        stds.append(score.std(ddof = 1))
    return means, stds

## Data import

In [3]:
raw_data = pd.read_csv('Data.csv')

for i in range(3):
    for j in range(raw_data.shape[0]):
        raw_data.iloc[j,i+2] = np.float64(raw_data.iloc[j,i+2].replace(',',''))
        
for i in range(raw_data.shape[0]):
    for j in range(raw_data.shape[1]):
        if (pd.isna(raw_data.iloc[i,j])):
            nanfill(raw_data,i,j)

## Analysis of Correlation

In [ ]:
group = [['LACCESS_POP15','LACCESS_LOWI15','LACCESS_HHNV15','LACCESS_CHILD15','LACCESS_SENIORS15'],
        ['GROCPTH16', 'SUPERCPTH16', 'CONVSPTH16', 'SPECSPTH16', 'WICSPTH16'],
        ['FFRPTH16', 'FSRPTH16'],
        ['FOODINSEC_15_17', 'VLFOODSEC_15_17'],
        ['FMRKT_WIC18', 'FMRKT_WICCASH18'],
        ['POVRATE15', 'CHILDPOVRATE15']]

In [ ]:
for i in range(len(group)):
    corr = raw_data[group[i]].corr()
    plt.figure(figsize = (12,8), dpi = 300)
    sns.heatmap(corr,linewidths=0.1,vmax=1.0, square=True,linecolor='white', annot=True)
    plt.savefig('corr_heatmap_{}.jpg'.format(i))

## Data preprocessing

In [4]:
preserve_columns = ['State','LACCESS_POP15','GROCPTH16',
                    'SUPERCPTH16','CONVSPTH16','SPECSPTH16','WICSPTH16','FFRPTH16','FSRPTH16',
                    'FOODINSEC_15_17','FMRKT_WIC18','POVRATE15','PCT_WIC17']

raw_data['PCT_WIC17'] = raw_data['PCT_WIC17'] * raw_data['Population_Estimate_2016'] / 100
raw_data['FOODINSEC_15_17'] = raw_data['FOODINSEC_15_17'] * raw_data['Population_Estimate_2016'] / 100
raw_data['POVRATE15'] = raw_data['POVRATE15'] * raw_data['Population_Estimate_2016'] / 100

raw_data = raw_data[preserve_columns]


# for i in range(raw_data.shape[1]-2):
#     plt.figure(figsize = (12,8), dpi = 300)
#     plt.plot(range(raw_data.shape[0]), raw_data.iloc[:,i+2])
#     print(preserve_columns[i+2])



In [5]:
raw_data

,State,LACCESS_POP15,GROCPTH16,SUPERCPTH16,CONVSPTH16,SPECSPTH16,WICSPTH16,FFRPTH16,FSRPTH16,FOODINSEC_15_17,FMRKT_WIC18,POVRATE15,PCT_WIC17
0,Alabama,17496.693040,0.054271,0.018090,0.560802,0.018090,0.090511,0.795977,0.560802,9004.446,0.0,7015.734,1405.118725
1,Alabama,30561.264430,0.139753,0.033733,0.568650,0.130115,0.134802,0.751775,1.137300,33857.056,0.0,26794.848,5283.29931
2,Alabama,6069.523628,0.155195,0.038799,0.737177,0.077598,0.232387,0.892372,0.543183,4208.497,0.0,8262.08,656.724238
3,Alabama,969.378841,0.220916,0.044183,0.662749,0.000000,0.221474,0.309283,0.309283,3679.888,0.0,5011.872,574.236275
4,Alabama,3724.428242,0.086863,0.017373,0.469059,0.000000,0.139089,0.399569,0.208471,9375.271,1.0,8454.999,1462.984933
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3136,Wyoming,18934.737810,0.090406,0.022601,0.497231,0.022601,0.090344,0.700644,0.723246,5844.3,0.0,3763.375,776.384661
3137,Wyoming,6212.390430,0.474547,0.000000,0.819672,0.345125,0.129528,0.862813,2.545298,3057.252,0.0,1528.626,406.139924
3138,Wyoming,4686.017653,0.096567,0.048284,0.627686,0.096567,0.144991,0.820820,0.917387,2731.212,0.0,2027.718,362.827217
3139,Wyoming,931.411647,0.244260,0.000000,0.366390,0.000000,0.244858,0.732780,1.465559,1078.176,0.0,914.816,143.230038


In [5]:
temp = raw_data.iloc[:,1:]
normalization(temp)
raw_data.iloc[:,1:] = temp
enc = TargetEncoder(cols = ['State']).fit(raw_data['State'],raw_data['PCT_WIC17'])
raw_data['State'] = enc.transform(raw_data['State'])
data = raw_data.sample(frac = 1, random_state = 913).values
X_train = np.float64(data[:2512,1:-1])
y_train = np.float64(data[:2512,-1])
X_test = np.float64(data[2512:,1:-1])
y_test = np.float64(data[2512:,-1])

c:\Software\Conda\envs\py38\lib\site-packages\category_encoders\target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
c:\Software\Conda\envs\py38\lib\site-packages\category_encoders\target_encoder.py:127: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


## Baseline

In [10]:
models = []
for i in range(5):
    clf = LinearRegression()
    models.append(clf)
cross_validate(models)

([0.908892733654994,
  0.908892733654994,
  0.908892733654994,
  0.908892733654994,
  0.908892733654994],
 [0.025675379370049146,
  0.025675379370049146,
  0.025675379370049146,
  0.025675379370049146,
  0.025675379370049146])

## Ridge

In [11]:
Ridge_clfs = []
for i in range(5):
    clf = KernelRidge(alpha = 10 ** (i - 2), kernel = 'linear')
    Ridge_clfs.append(clf)
Ridge_means1, Ridge_stds1 = cross_validate(Ridge_clfs)

In [12]:
Ridge_clfs = []
for i in range(5):
    clf = KernelRidge(alpha = 10 ** (i - 2), kernel = 'sigmoid')
    Ridge_clfs.append(clf)
Ridge_means2, Ridge_stds2 = cross_validate(Ridge_clfs)

c:\Software\Conda\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:251: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
c:\Software\Conda\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:251: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
c:\Software\Conda\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:251: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
c:\Software\Conda\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:251: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
c:\Software\Conda\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:251: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
c:\Software\Conda\envs\py38\lib\site-packages

In [13]:
Ridge_clfs = []
for i in range(5):
    clf = KernelRidge(alpha = 10 ** (i - 2), kernel = 'rbf')
    Ridge_clfs.append(clf)
Ridge_means3, Ridge_stds3 = cross_validate(Ridge_clfs)

In [14]:
Ridge_means1,Ridge_means2,Ridge_means3

([0.908996591128048,
  0.909013209198244,
  0.9091768733499028,
  0.9105964125778719,
  0.9160827473828184],
 [-301.6674032159934,
  -485.71645232535894,
  -10.515867227142687,
  -23.512478574983252,
  0.042959089447886914],
 [0.4452518915240763,
  0.43894749255387067,
  0.3916969051431077,
  0.2757141053532872,
  0.13362412321627243])

## Lasso

In [15]:
Lasso_clfs = []
for i in range(5):
    clf = Lasso(alpha = 10 ** (i - 2), max_iter = 5000, random_state = 913)
    Lasso_clfs.append(clf)
Lasso_mean, Lasso_std = cross_validate(Lasso_clfs)
Lasso_mean

[0.9132022195071137,
 0.9102684539827521,
 0.10137692798669244,
 -0.001469115496448703,
 -0.001469115496448703]

## Elastic Net

In [16]:
ElasticNet_clfs = []
l1_ratio = [0.25,0.5,0.75]
for i in range(3):
    same_ratio = []
    for j in range(5):
        clf = ElasticNet(alpha = 10 ** (i - 2), l1_ratio = l1_ratio[i], random_state = 913, max_iter = 5000)
        same_ratio.append(clf)
    ElasticNet_clfs.append(same_ratio)

EN_means = []
EN_stds = []
for i in range(3):
    EN_mean, EN_std = cross_validate(ElasticNet_clfs[i])
    EN_means.append(EN_mean)
    EN_stds.append(EN_std)

In [17]:
EN_means

[[0.9124657160524936,
  0.9124657160524936,
  0.9124657160524936,
  0.9124657160524936,
  0.9124657160524936],
 [0.9196284480440952,
  0.9196284480440952,
  0.9196284480440952,
  0.9196284480440952,
  0.9196284480440952],
 [0.36707347695535114,
  0.36707347695535114,
  0.36707347695535114,
  0.36707347695535114,
  0.36707347695535114]]

## Decision tree

In [18]:
DecisionTrees = []
clf = DecisionTreeRegressor(criterion = 'squared_error', random_state = 913)
DecisionTrees.append(clf)
clf = DecisionTreeRegressor(criterion = 'friedman_mse', random_state = 913)
DecisionTrees.append(clf)
clf = DecisionTreeRegressor(criterion = 'absolute_error', random_state = 913)
DecisionTrees.append(clf)
# clf = DecisionTreeRegressor(criterion = 'poisson', random_state = 913)
# DecisionTrees.append(clf)
cross_validate(DecisionTrees)

([0.784789325702335, 0.8234174701844861, 0.7624839242475426],
 [0.16170174923836303, 0.05645898434575921, 0.15171839995927175])

## NN

In [21]:
%%time
NNs = []
size_list = [(64,8),(64,16,4),(128,16),(128,32,8)]
for i in range(len(size_list)):
    NN_same_size = []
    for j in range(3):
        clf = MLPRegressor(hidden_layer_sizes = size_list[i], learning_rate_init = 10 ** (-i - 2),
                           random_state = 913, max_iter = 2000)
        NN_same_size.append(clf)
    NNs.append(NN_same_size)

NN_means = []
NN_stds = []
for i in range(len(NNs)):
    NN_mean, NN_std = cross_validate(NNs[i])
    NN_means.append(NN_mean)
    NN_std.append(NN_std)

CPU times: total: 6min 8s
Wall time: 4min 7s


In [22]:
NN_means

[[0.7633080957089098, 0.7633080957089098, 0.7633080957089098],
 [0.9100688460208854, 0.9100688460208854, 0.9100688460208854],
 [0.9174429995000721, 0.9174429995000721, 0.9174429995000721],
 [0.8800609478430304, 0.8800609478430304, 0.8800609478430304]]